# Inside Traffic4castDataset __init__

In [1]:
import sys
sys.path.append("../Traffic4cast")

In [2]:
from functools import partial
import pickle

import torch

In [3]:
import t4c22
from t4c22.t4c22_config import cc_dates
from t4c22.dataloading.road_graph_mapping import TorchRoadGraphMapping
from t4c22.t4c22_config import load_basedir
from t4c22.t4c22_config import day_t_filter_to_df_filter
from t4c22.t4c22_config import day_t_filter_weekdays_daytime_only

In [19]:
root = load_basedir(fn="t4c22_config.json", pkg=t4c22)
cachedir = "../data/cache"
split = "train"
fill = 1
normalize = "zs"
city = "london"
limit = None
day_t_filter = day_t_filter_weekdays_daytime_only
edge_attributes=["speed_kph", "parsed_maxspeed", "length_meters", "counter_distance",
     'num_lanes', 'mean_pop', 'catering', 'leisure', 'shopping', 'tourism', 'traffic', 'transport',
    'pois_total', 'greeneries', 'water_bodies', 'built_up', 'other', "importance", "highway", "oneway",]
enriched = True

In [20]:
torch_road_graph_mapping = TorchRoadGraphMapping(
    city=city,
    edge_attributes=edge_attributes,
    root=root,
    enriched=enriched,
    df_filter=partial(day_t_filter_to_df_filter,
                      filter=day_t_filter)
)

In [21]:
torch_road_graph_mapping.edge_attr.shape

torch.Size([132414, 20])

In [29]:
torch_road_graph_mapping.edge_attr[:, -1].unique() # oneway

tensor([0., 1.])

In [30]:
torch_road_graph_mapping.edge_attr[:, -2].unique() # highway

tensor([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.])

In [31]:
torch_road_graph_mapping.edge_attr[:, -3].unique() # importance

tensor([0., 1., 2., 3., 4., 5.])

In [32]:
# `day_t: List[Tuple[Y-m-d-str,int_0_96]]`
# TODO most days have even 96 (rolling window over midnight), but probably not necessary because of filtering we do.
if split == "test":
    num_tests = load_inputs(basedir=root, split="test", city=city, day="test", df_filter=None)[
        "test_idx"].max() + 1
    day_t = [("test", t) for t in range(num_tests)]
else:
    # TODO: discard slicing
    day_t = [(day, t) for day in cc_dates(root, city=city, split=split) for t in range(4, 96) if
                  day_t_filter(day, t)]

In [33]:
len(day_t)

7040

In [34]:
cluster_map = {}
with open(root / f"{city}.pkl", 'rb') as f:
    maps = pickle.load(f)
for i in range(20):
    for day, t, _ in maps[i]:
        cluster_map[f"{day}-{t}"] = i
if (len(day_t) == 0):
    print("no sample")
    exit(0)

In [35]:
len(cluster_map)

7140

In [36]:
edge_index = torch_road_graph_mapping.edge_index

In [12]:
# edge_attr = get_edge_attr()

## Break down get_edge_attr method

In [37]:
def minmax(x, min_v, max_v):
    x = x - min_v
    x = x / (max_v - min_v)
    x = x * 2 - 1
    return x

In [38]:
# edge_attributes = ["speed_kph", "parsed_maxspeed", "length_meters", "counter_distance", "importance", "highway", "oneway", ]
num_edge_attributes = torch_road_graph_mapping.edge_attr.shape[1]
num_edge_attributes

20

In [42]:
num_importance = torch.max(
    torch_road_graph_mapping.edge_attr[:, -3]).item() + 1
num_importance

6.0

In [43]:
num_highway = torch.max(
    torch_road_graph_mapping.edge_attr[:, -2]).item() + 1
num_highway

12.0

In [44]:
num_oneway = torch.max(
    torch_road_graph_mapping.edge_attr[:, -1]).item() + 1
num_oneway

2.0

In [45]:
edge_attr = torch_road_graph_mapping.edge_attr[:, :num_edge_attributes-3]
edge_attr.shape

torch.Size([132414, 17])

In [46]:
edge_attr = minmax(edge_attr, torch.min(edge_attr, dim=0).values, torch.max(edge_attr, dim=0).values)
edge_attr.shape

torch.Size([132414, 17])

In [47]:
edge_importance = torch.zeros([edge_attr.shape[0], int(num_importance)], dtype=torch.float)
edge_importance.shape

torch.Size([132414, 6])

In [48]:
edge_highway = torch.zeros([edge_attr.shape[0], int(num_highway)], dtype=torch.float)
edge_highway.shape

torch.Size([132414, 12])

In [49]:
edge_oneway = torch.zeros([edge_attr.shape[0], int(num_oneway)], dtype=torch.float)
edge_oneway.shape

torch.Size([132414, 2])

In [51]:
edge_importance[
    [i for i in range(edge_attr.shape[0])], [int(j) for j in torch_road_graph_mapping.edge_attr[:, num_edge_attributes-3]]] = 1
edge_highway[
    [i for i in range(edge_attr.shape[0])], [int(j) for j in torch_road_graph_mapping.edge_attr[:, num_edge_attributes-2]]] = 1
edge_oneway[
    [i for i in range(edge_attr.shape[0])], [int(j) for j in torch_road_graph_mapping.edge_attr[:, num_edge_attributes-1]]] = 1

In [52]:
torch.cat([edge_attr, edge_importance, edge_highway, edge_oneway], dim=1).shape

torch.Size([132414, 37])